In [3]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

### Agent With SearchAPI

In [2]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_community.utilities import SearchApiAPIWrapper
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0, model_name="gpt-4-turbo")
search = SearchApiAPIWrapper()

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
self_ask_with_search.run("قضیه اعتصاب طلافروشان تهران چیست؟")

### Search With DuckDuckGo

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(backend="news")
print(search.run("امروز قیمت طلا چنده؟"))

### LLMChain with Tavily Search API

In [5]:
from langchain_community.retrievers import TavilySearchAPIRetriever

from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
retriever = TavilySearchAPIRetriever(k=3)

prompt = ChatPromptTemplate.from_template(
"""Answer the question based only on the context provided.

Context: {context}

Question: {question}
"""
)

chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-turbo", verbose=True)
    | StrOutputParser()
)

In [ ]:
chain.invoke(
    {"question": "الان قیمت طلا گرمی چنده؟"},
    config={'callbacks': [ConsoleCallbackHandler()]}
)

### Custom Agent With Tavily Search API

In [60]:
from datetime import datetime
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain.callbacks import get_openai_callback

In [44]:
class CurrentDatetimeTool(BaseTool):
    def __init__(self):
        super().__init__(name="current_datetime", description="Returns the current datetime")

    def _run(self, *args, **kwargs):
        return {"datetime": datetime.now().isoformat()}

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("This tool does not support async")

In [45]:
tools = [TavilySearchResults(max_results=1), CurrentDatetimeTool()]

In [30]:
base_prompt = hub.pull("langchain-ai/openai-functions-template")

In [31]:
base_prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['instructions'], template='{instructions}')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [53]:
instructions = """
You are a helpful assistant. Your task is to understand the user's question and provide accurate, factual answers. If the question requires worldwide or updated data, use the search and date-time functions to obtain the necessary information. For questions that require comparisons, query data for each statement as needed.

Follow these steps to answer a question:
1. Read the question.
2. Thoroughly analyze the question.
3. Break down long questions into sub-questions.
4. Query data for sub-questions if your existing knowledge is insufficient.
5. Provide a detailed answer based on the data.

Query Data Note:
- For time-related data, use the 'current_datetime' function to obtain the current date and time, and include this information in your query.
- If you cannot find the needed data or sufficient information, mention a reference or source where the user can find the data themselves.
"""

prompt = base_prompt.partial(instructions=instructions)

In [75]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o")

# Construct the OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [80]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, stream_runnable=False, verbose=True)

In [79]:
with get_openai_callback() as cb:
    result = agent_executor.invoke({"input": "امروز قیمت دلار چند تومان است؟"})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    total_cost = (cb.prompt_tokens*0.005 + cb.completion_tokens*0.015) / 1000
    print(f"Total Cost (USD): ${total_cost:.6f}")



> Entering new AgentExecutor chain...

Invoking: `current_datetime` with `current date`


{'datetime': '2024-05-19T12:57:05.639338'}
Invoking: `tavily_search_results_json` with `{'query': 'قیمت دلار امروز ۱۹ می ۲۰۲۴'}`


[{'url': 'https://tejaratnews.com/بخش-طلا-ارز-14/901488-قیمت-دلار-امروز-اسفند', 'content': 'پیش بینی قیمت دلار ۱۹ اسفند ۱۴۰۲ / دلار به کریدور ۵۰ هزار تومانی برمی\u200cگردد؟. قیمت دلار در نخستین روز هفته با روند صعودی بازگشایی شد. علاوه بر رشد تقاضا، افزایش قیمت در بازارهای جهانی و صعود ...'}]برای دریافت دقیق‌ترین و به‌روزترین قیمت دلار، پیشنهاد می‌کنم به سایت‌های معتبر مالی و خبری مانند "تجارت‌نیوز" یا "خبرگزاری‌های رسمی" مراجعه کنید. در حال حاضر، اطلاعات دقیق قیمت دلار برای تاریخ ۱۹ می ۲۰۲۴ در دسترس نیست.

> Finished chain.
Total Tokens: 1222
Prompt Tokens: 1112
Completion Tokens: 110
Total Cost (USD): $0.007210


In [78]:
from IPython.display import display, Markdown

display(Markdown(result["output"]))

برای مشاهده قیمت دقیق دلار به روز ۱۹ می ۲۰۲۴، می‌توانید به وبسایت‌های معتبر مانند [شبکه اطلاع رسانی طلا، سکه و ارز](https://www.tgju.org/) مراجعه کنید. در این وبسایت‌ها قیمت زنده دلار و سایر ارزها به صورت لحظه‌ای به‌روز می‌شود.